In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import ipdb

In [2]:
# mapping between abbreviated id, pedigree, male and female
workbook = 'HDP_MapFL18.xlsx'
sheet = 'ID_conversion'
mapping_df = pd.read_excel(workbook, sheet_name=sheet)
# list of all row indices which match the criteria
# idx = mapping_df.index[df['ID_abbreviated'] == '18R025'].tolist()
# mapping_df.iloc[idx[0]]['Male']

def get_parents(mapping_df, females_self_pollinating, females, hybrid):
    if hybrid == 'FILL' or hybrid == 'CHECK':
        return None, None
    
#     if hybrid == '83P17':
#         ipdb.set_trace()
    # there are some genotypes of only female
    if hybrid in females_self_pollinating:
        return None, females[females_self_pollinating.index(hybrid)]
    
    # return male and female of the parent hybrid plant
    idx = mapping_df.index[mapping_df['ID_abbreviated']==hybrid].tolist()
    idx1 = mapping_df.index[mapping_df['ID_abbreviated']=='PI'+hybrid].tolist()
    if len(idx) == 0 and len(idx1)==0:
        return None, None
    elem = idx[0] if len(idx1)==0 else idx1[0]
    male = mapping_df.iloc[elem]['Male']
    female = mapping_df.iloc[elem]['Female']
    if male == np.nan:
        male = None
    if female != female:
        female = None
    return male, female


In [3]:
# NOTE: not needed anymore
# the raw data files (originally csv files) are now stored in raw_data directory.

# data_dirs = '/home/sumit/old_stuffs/nifa/2018_field_season/ground_robot_phenotyping/30DAP/Phenotypes/'
# import os
# all_files = os.listdir(data_dirs)
# for f in all_files:
#     if f=='height_ground_robot_measures.csv':
#         dlim = ','
#     else:
#         dlim = '\t+'
#     df = pd.read_csv(data_dirs+f, delimiter=dlim, engine='python')
#     df.to_pickle('raw_data/'+f.split('.')[0] + '.pkl')

In [4]:
def get_mask(vec, ignores=[]):
    # False means the value is nan or one of the ignores
    mask = np.full(len(vec), True)
    for i, v in enumerate(vec):
        if v is np.nan or v in ignores:
            mask[i] = False
    return mask

data = mapping_df.values
columns = list(mapping_df)
male = data[:, columns.index('Male')]
female = data[:, columns.index('Female')]
hybrid = data[:, columns.index('ID_abbreviated')]
ignores = ['Check']
male_mask = get_mask(male, ignores)
female_mask = get_mask(female, ignores)
hybrid_mask = get_mask(hybrid, ignores)

# male mask and female mask are important, not sure about hybrid mask though
print(len(np.unique(female[female_mask])))
print(len(np.unique(male[male_mask])))
print(len(np.unique(hybrid[hybrid_mask])))

np.unique(female[female_mask])

# not sure if this genotype is male/female (it is in the females column though)
# but doesn't seem to have been crossed with any male
unknowns = ['83P17']
females_geno = ['ATx623', 'ATx642', 'ATxARG-1', 'PHA86']
females_self_pollinating = ['BTx623', 'BTx642', 'BTxARG-1', 'PHB86']

4
42
210


In [5]:
data_file = 'raw_data/height_ground_robot_measures.pkl'
f_df = pd.read_pickle(data_file)

# make a dataframe with columns as Row1 Row2 Range ID_abbreviated Male Female
hybrids = f_df[['ID_abbreviated']].values.squeeze()
males = []
females = []
for h in hybrids:
    m, f = get_parents(mapping_df, females_self_pollinating + unknowns, females_geno + unknowns, h)
    males.append(m)
    females.append(f)
    
males = np.array(males)
females = np.array(females)

males_none = np.where(males==None)
males_none_hybrids = np.unique(hybrids[males_none])
# males None correponds to these hybrids -> ['83P17', 'BTx623', 'BTx642', 'BTxARG-1', 'FILL', 'PHB86']
# among which everyone except '83P17' (don't know what is this) is female. 

all_males = np.unique(males[males!=None])
count_males = [np.sum(males==m) for m in all_males]
# a small test to determine if the hybrid to male detection is correct or not
# for h, m in zip(hybrids, male):
#     if m is None and h not in females_self_pollinating + unknowns + ['FILL']:
#         print(h,m)

all_females = np.unique(females[females!=None])
count_females = [np.sum(females==f) for f in all_females]
print(all_males)
print(count_males)
print(all_females)
print(count_females)

['FL17_116' 'PI533752' 'PI533755' 'PI533757' 'PI533761' 'PI533769'
 'PI533788' 'PI533789' 'PI533794' 'PI533822' 'PI533855' 'PI533882'
 'PI533961' 'PI533972' 'PI533974' 'PI533976' 'PI533987' 'PI533989'
 'PI533998' 'PI534050' 'PI534053' 'PI534112' 'PI534125' 'PI534146'
 'PI564165' 'PI576349' 'PI576426' 'PI585295' 'PI595699' 'PI595721'
 'PI595740' 'PI597944' 'PI597950' 'PI613536' 'PI629034' 'PI655996'
 'PI656009' 'PI656023' 'PI656046' 'PI656056' 'PI656078' 'PV17_224']
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 8, 10, 10, 10, 10, 10, 8, 10, 9, 10, 10, 10, 10, 10, 9, 10, 10]
['83P17' 'ATx623' 'ATx642' 'ATxARG-1' 'PHA86']
[12, 96, 92, 94, 95]


In [18]:
# read each raw_data file and get stats of each male genotype
phenotype = 'plant_count_measures'
print(phenotype)
df = pd.read_pickle('raw_data/' + phenotype + '.pkl')
def get_stats(values, all_males):
    dct = {}
    for m in all_males:
        if m is not None:
            dct['m'] = []
        
    for i in range(len(values)):
        if all_males[i] is not None:
            dct[all_males[i]].append(values[i])
    return dct

df

plant_count_measures


,Row1,Row2,Range,Mean,Variance
0,2.0,3.0,1.0,12.918170,18.346411
1,2.0,3.0,2.0,7.721130,40.326385
2,2.0,3.0,3.0,11.224403,29.348141
3,2.0,3.0,4.0,9.149849,48.209057
4,2.0,3.0,5.0,8.388241,29.044528
5,2.0,3.0,6.0,8.708267,31.711587
6,2.0,3.0,7.0,6.673006,24.540115
7,2.0,3.0,8.0,10.214106,30.979957
8,2.0,3.0,9.0,10.273721,28.815232
9,2.0,3.0,10.0,7.856417,33.042484


In [84]:
count_male = [(male==m).sum() for m in np.unique(male[male_mask])]
count_female = [(female==f).sum() for f in np.unique(female[female_mask])]
count_hybrid = [(hybrid==h).sum() for h in np.unique(hybrid[hybrid_mask])]

In [7]:
# group males (there are 42 of them) together using one phenotype
all_males

array(['FL17_116', 'PI533752', 'PI533755', 'PI533757', 'PI533761',
       'PI533769', 'PI533788', 'PI533789', 'PI533794', 'PI533822',
       'PI533855', 'PI533882', 'PI533961', 'PI533972', 'PI533974',
       'PI533976', 'PI533987', 'PI533989', 'PI533998', 'PI534050',
       'PI534053', 'PI534112', 'PI534125', 'PI534146', 'PI564165',
       'PI576349', 'PI576426', 'PI585295', 'PI595699', 'PI595721',
       'PI595740', 'PI597944', 'PI597950', 'PI613536', 'PI629034',
       'PI655996', 'PI656009', 'PI656023', 'PI656046', 'PI656056',
       'PI656078', 'PV17_224'], dtype=object)